<a href="https://colab.research.google.com/github/nanpolend/machine-learning/blob/master/%E9%90%B5%E9%81%94%E5%B0%BC%E8%99%9F%E7%94%9F%E5%AD%98%E9%A0%90%E6%B8%AC_deepseek%E4%BF%AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import requests
import io
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
import pandas as pd

# 架構圖:
# 1. 資料獲取: 從 URL 下載 titanic.csv 檔案
# 2. 資料預處理:
#    - 使用 StringIO 將檔案內容轉換為檔案物件
#    - 使用 pandas 讀取資料
#    - 處理缺失值: 填充年齡缺失值、移除必要特徵的缺失值
#    - 特徵工程: 使用 LabelEncoder 編碼 Pclass 特徵
# 3. 模型訓練:
#    - 創建特徵矩陣 X 和目標變量 y
#    - 初始化 LogisticRegression 模型
#    - 使用訓練資料訓練模型
# 4. 模型評估:
#    - 使用模型進行預測
#    - 計算混淆矩陣、準確率等評估指標

# 下載 titanic.csv 檔案
url = "https://calmcode.io/static/data/titanic.csv"
response = requests.get(url)
titanic_data = response.content

# 使用 StringIO 將檔案內容轉換為檔案物件
titanic_file = io.StringIO(titanic_data.decode('utf-8'))

# 讀取資料並處理缺失值
titanic = pd.read_csv(titanic_file)
# Print the column names to verify the correct name for the age column
print(titanic.columns)
# Access the age column using the correct name, for instance, if the column name is 'age' instead of 'Age'
titanic["age"] = titanic["age"].fillna(titanic["age"].median())  # 填充年齡缺失值
# Assuming 'sex' and 'pclass' are the correct column names based on the URL
titanic = titanic.dropna(subset=["pclass", "sex"])  # 移除必要特徵的缺失值

# 特徵工程
label_encoder = preprocessing.LabelEncoder()
encoded_class = label_encoder.fit_transform(titanic["pclass"])

# Add a new column for encoded sex
titanic["sex_code"] = label_encoder.fit_transform(titanic["sex"])

# 創建特徵矩陣（使用字典方式更清晰）
X = pd.DataFrame({
    "Pclass": encoded_class,
    "SexCode": titanic["sex_code"],  # Use the new encoded sex column
    "Age": titanic["age"]
})

# 目標變量
y = titanic["survived"]

# 初始化模型
logistic = LogisticRegression(max_iter=1000)  # 增加迭代次數確保收斂

# 訓練模型
logistic.fit(X, y)

# 輸出模型參數
print("迴歸係數:", logistic.coef_)
print("截距:", logistic.intercept_)

# 模型評估
preds = logistic.predict(X)
print("\n混淆矩陣:")
print(confusion_matrix(y, preds))
print("\n模型評估指標:")
print(f"準確率: {accuracy_score(y, preds):.2f}")
print(f"訓練集樣本數: {len(X)}")
print(f"特徵數量: {X.shape[1]}")
print(f"類別分布:\n{y.value_counts()}")

Index(['survived', 'pclass', 'name', 'sex', 'age', 'fare', 'sibsp', 'parch'], dtype='object')
迴歸係數: [[-1.24883444 -2.4123266  -0.03611812]]
截距: [3.63192566]

混淆矩陣:
[[357  67]
 [ 83 207]]

模型評估指標:
準確率: 0.79
訓練集樣本數: 714
特徵數量: 3
類別分布:
survived
0    424
1    290
Name: count, dtype: int64
